In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
import pandas as pd
import os
import re
import string
import pickle
from transformers import BertForSequenceClassification
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import numpy as np
from sklearn.model_selection import train_test_split

from vecsim_app.categories import CATEGORIES
from vecsim_app.data_utils import papers

DATA_PATH = "/home/jovyan/arxiv/arxiv-metadata-oai-snapshot.json"
YEAR_CUTOFF = 2012
YEAR_PATTERN = r"(19|20[0-9]{2})"

In [4]:
df = pd.DataFrame(papers(data_path=DATA_PATH, year_cutoff=YEAR_CUTOFF, year_pattern=YEAR_PATTERN))
len(df)

409500

In [5]:
df.head(3)

,id,title,year,authors,categories,abstract
0,0704.0304,The World as Evolving Information,2012,Carlos Gershenson,"cs.IT,cs.AI,math.IT,q-bio.PE",This paper discusses the benefits of describ...
1,0704.2744,Nahm transform and parabolic minimal Laplace t...,2012,Szilard Szabo,math.AG,We prove that Nahm transform for integrable ...
2,0704.2768,Heat Equations and the Weighted $\bar\partial$...,2012,Andrew Raich,"math.AP,math.CV",The purpose of this article is to establish ...


In [6]:
df['text'] = df['title'] + ' ' + df['abstract']
# df['categories'] = df['categories'].apply(lambda x: x.split(','))

In [7]:
df.iloc[0].categories

'cs.IT,cs.AI,math.IT,q-bio.PE'

In [8]:
# Split into train and test
df_2, df_unused = train_test_split(df, train_size=0.6)  # take only 60% of dataset
df_train, df_test = train_test_split(df_2, train_size=0.8)

df.size, df_train.size, df_test.size

(2866500, 1375920, 343980)

In [9]:
def get_tokenizer(tokenizer_model):
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)
    return tokenize_function, tokenizer

tokenize_function, tokenizer = get_tokenizer('bert-base-uncased')

In [10]:
mlb = MultiLabelBinarizer()
# mlb.fit([[(k,v) for k, v in CATEGORIES.items()]]) #df_train['categories'])
mlb.fit([list(CATEGORIES.keys())]) #df_train['categories'])
mlb.classes_[:10]

array(['astro-ph', 'astro-ph.CO', 'astro-ph.EP', 'astro-ph.GA',
       'astro-ph.HE', 'astro-ph.IM', 'astro-ph.SR', 'cond-mat.dis-nn',
       'cond-mat.mes-hall', 'cond-mat.mtrl-sci'], dtype=object)

In [11]:
def preprocess_data(examples):
    # take a batch of texts
    text = examples["text"]

    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)

    encoded_categories = mlb.transform([c.split(',') for c in examples['categories']]).astype(float)

    encoding["labels"] = encoded_categories

    return encoding

In [12]:
df_train_hf = Dataset.from_pandas(df_train[['text', 'categories']])
tokenized_train = df_train_hf.map(preprocess_data, batched=True)

df_test_hf = Dataset.from_pandas(df_test[['text', 'categories']])
tokenized_test = df_test_hf.map(preprocess_data, batched=True)

  0%|          | 0/197 [00:00<?, ?ba/s]

/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.GN', 'eess.SY'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['eess.SY'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.GN', 'econ.TH', 'eess.SY'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.TH', 'eess.SY'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['chao-dyn', 'econ.GN', 'econ.TH', 'eess.SY'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: u

  0%|          | 0/50 [00:00<?, ?ba/s]

/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['eess.SY'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.GN', 'eess.SY'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.TH', 'eess.SY'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.GN', 'econ.TH', 'eess.SY'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.GN'] will be ignored
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['cond-mat', 'eco

In [13]:
# Debugging - get inverse transform

print("Reversed", mlb.inverse_transform(np.asarray(tokenized_test[0]['labels']).reshape(1, -1)))
print("Original categories", tokenized_test[0]['categories'])

Reversed [('quant-ph',)]
Original categories quant-ph


In [14]:
!rm -r checkpoint
!mkdir checkpoint
with open('checkpoint/mlb.pkl', 'wb') as handle:
    pickle.dump(mlb, handle, protocol=pickle.HIGHEST_PROTOCOL)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Training multi label class model

In [15]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels=len(mlb.classes_), 
    problem_type="multi_label_classification"
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [16]:
# Adaptation: https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb 
# 
batch_size = 75
metric_name = "f1"

args = TrainingArguments(
    f"paper-multilabel-finetuning",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size//2,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    eval_accumulation_steps=1,
)


# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    return multi_label_metrics(
        predictions=p.predictions, 
        labels=p.label_ids)

In [17]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [18]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text, categories. If __index_level_0__, text, categories are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/srv/conda/envs/saturn/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 196560
  Num Epochs = 5
  Instantaneous batch size per device = 75
  Total train batch size (w. parallel, distributed & accumulation) = 75
  Gradient Accumulation steps = 1
  Total optimization steps = 13105
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` met

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.047000,0.042835,0.092858,0.524497,0.044444
2,0.031400,0.029638,0.480396,0.670524,0.255881
3,0.026100,0.025347,0.563034,0.716737,0.312576
4,0.023600,0.023715,0.591987,0.734537,0.336264
5,0.022300,0.023287,0.599795,0.739657,0.343000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text, categories. If __index_level_0__, text, categories are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 49140
  Batch size = 37
Saving model checkpoint to paper-multilabel-finetuning/checkpoint-2621
Configuration saved in paper-multilabel-finetuning/checkpoint-2621/config.json
Model weights saved in paper-multilabel-finetuning/checkpoint-2621/pytorch_model.bin
tokenizer config file saved in paper-multilabel-finetuning/checkpoint-2621/tokenizer_config.json
Special tokens file saved in paper-multilabel-finetuning/checkpoint-2621/special_tokens_map.json
Deleting older checkpoint [paper-multilabel-finetuning/checkpoint-33] due to args.save_total_limit
Deleting older checkpoint [paper-multilabel-finetuning/checkpoint-165] due to ar

TrainOutput(global_step=13105, training_loss=0.03672803733604457, metrics={'train_runtime': 23459.4237, 'train_samples_per_second': 41.894, 'train_steps_per_second': 0.559, 'total_flos': 6.47340318084096e+16, 'train_loss': 0.03672803733604457, 'epoch': 5.0})

In [19]:
eval = trainer.evaluate()
eval

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text, categories. If __index_level_0__, text, categories are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 49140
  Batch size = 37


{'eval_loss': 0.023286648094654083,
 'eval_f1': 0.5997945229744571,
 'eval_roc_auc': 0.7396573760806298,
 'eval_accuracy': 0.342999592999593,
 'eval_runtime': 360.8244,
 'eval_samples_per_second': 136.188,
 'eval_steps_per_second': 3.683,
 'epoch': 5.0}

In [20]:
text = df['text'].iloc[5]
categories = df['categories'].iloc[5]
print(categories)

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k, v in encoding.items()}

outputs = trainer.model(**encoding)
logits = outputs.logits

cond-mat.stat-mech,cond-mat.other


In [21]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.45)] = 1

In [22]:
print(text)
print(mlb.inverse_transform(predictions.reshape(1, -1)))

Suppression of growth by multiplicative white noise in a parametric
  resonant system   The author studied the growth of the amplitude in a Mathieu-like equation
with multiplicative white noise. The approximate value of the exponent at the
extremum on parametric resonance regions was obtained theoretically by
introducing the width of time interval, and the exponents were calculated
numerically by solving the stochastic differential equations by a symplectic
numerical method. The Mathieu-like equation contains a parameter $\alpha$ that
is determined by the intensity of noise and the strength of the coupling
between the variable and the noise. The value of $\alpha$ was restricted not to
be negative without loss of generality. It was shown that the exponent
decreases with $\alpha$, reaches a minimum and increases after that. It was
also found that the exponent as a function of $\alpha$ has only one minimum at
$\alpha \neq 0$ on parametric resonance regions of $\alpha = 0$. This minimum
va

In [23]:
trainer.save_model(output_dir='./checkpoint')

Saving model checkpoint to ./checkpoint
Configuration saved in ./checkpoint/config.json
Model weights saved in ./checkpoint/pytorch_model.bin
tokenizer config file saved in ./checkpoint/tokenizer_config.json
Special tokens file saved in ./checkpoint/special_tokens_map.json


In [24]:
with open('./checkpoint/model_info.json', 'w') as f:
    f.write(json.dumps(eval, indent=4))